In [1]:
import numpy as np

In [2]:
np.random.seed(0)

In [3]:
data = "hello world"
chars = list(set(data))
vocab_size = len(chars)

chars_to_indices = {char: i for i, char in enumerate(chars)}
indices_to_chars = {i: char for i, char in enumerate(chars)}

chars_to_indices, indices_to_chars

({'d': 0, 'h': 1, 'r': 2, 'l': 3, 'w': 4, 'o': 5, 'e': 6, ' ': 7},
 {0: 'd', 1: 'h', 2: 'r', 3: 'l', 4: 'w', 5: 'o', 6: 'e', 7: ' '})

In [4]:
# RNN configs
embedding_size = vocab_size
hidden_size = 64
output_size = vocab_size

In [5]:
# Matrices
Wxh = np.random.randn(hidden_size, embedding_size)
Whh = np.random.randn(hidden_size, hidden_size)
bh = np.random.randn(1, hidden_size)

Why = np.random.randn(output_size, hidden_size)
by = np.random.randn(1, output_size)

In [6]:
def softmax(z):
    e_z = np.exp(z)
    return e_z / e_z.sum(axis=1)

def rnn(x_t, h_prev):
    assert x_t.shape == (1, embedding_size)
    assert h_prev.shape == (1, hidden_size)

    h_next = np.tanh(x_t @ Wxh.T + h_prev @ Whh.T + bh)
    probs = softmax(h_next @ Why.T + by)
    
    return probs, h_next

def sample(char, n):
    # sample rnn n times starting with the first char
    x = np.zeros((1, vocab_size))
    idx = chars_to_indices[char]
    x[:, idx] = 1 # one hot encoding
    h =  np.zeros((1, hidden_size)) # hidden state
    idxes = []
    
    for i in range(n):
        probs, h = rnn(x, h) # xt, hprev -> rnn -> probs, hnext
        idx = np.random.choice(vocab_size, p=probs.ravel()) # .ravel returns a 1d array
        
        x = np.zeros((1, vocab_size))
        x[:, idx] = 1
        idxes.append(idx)

    chars = "".join([indices_to_chars[i] for i in idxes])
    return chars

In [7]:
x_t = np.random.randn(1, embedding_size)
h_prev = np.random.randn(1, hidden_size)

probs, h_next = rnn(x_t, h_prev)
probs.shape, h_next.shape

((1, 8), (1, 64))

In [8]:
inp = [chars_to_indices[c] for c in data[:-1]]
target = [chars_to_indices[c] for c in data[1:]]

inp, target

([1, 6, 3, 3, 5, 7, 4, 5, 2, 3], [6, 3, 3, 5, 7, 4, 5, 2, 3, 0])

In [9]:
sample("h", 4)

'howw'

In [15]:
lr = 1e-2
for e in range(100):
    # compute loss
    loss = 0
    xs, ps, hs = {}, {}, {}
    h = np.zeros((1, hidden_size))
    hs[-1] = h
    for t in range(len(inp)):
        idx = inp[t]
        x = np.zeros((1, vocab_size))
        x[:, idx] = 1
        probs, h = rnn(x, h)
        xs[t] = x
        ps[t] = probs # save probs (we'll use this in backprop)
        hs[t] = h

        # cross entropy
        pred = probs[0, target[t]]
        loss += -np.log(pred)

    # backprop, calculate gradients
    dL_dWxh = np.zeros_like(Wxh)
    dL_dWhh = np.zeros_like(Whh)
    dL_dbh = np.zeros_like(bh)
    dL_Why = np.zeros_like(Why)
    dL_dby = np.zeros_like(by)
    dF_dh = np.zeros((1, hidden_size))

    for t in reversed(range(len(inp))):
        dL_dz2 = np.copy(ps[t])
        target_idx = target[t]
        dL_dz2[:, target_idx] -= 1

        # print(dL_dz2.shape)

        a = hs[t] * dL_dz2.T
        # print(a.shape)

        # 2nd layer
        dL_Why += hs[t] * dL_dz2.T
        dL_dby += dL_dz2


        # 1st layer
        dh_dz1 = 1 - hs[t] ** 2
        dL_dh = dF_dh + (dL_dz2 @ Why) 

        # print((xs[t] * dh_dz1.T * dL_dh.T).shape)

        dL_dWxh += xs[t] * dh_dz1.T * dL_dh.T
        dL_dWhh +=  hs[t - 1] * dh_dz1.T * dL_dh.T
        dL_dbh +=  dh_dz1 * dL_dh

        # raise RuntimeError

        dF_dh = dL_dh * (dh_dz1 @ Whh)

    # clip gradients
    for gradient in [dL_dWxh, dL_dWhh, dL_Why, dL_dbh, dL_dby]:
        np.clip(gradient, -1, 1, out=gradient)
        # np.clip(gradient, -5, 5, out=gradient)

    # gradient descent
    for weights, gradient in zip([Wxh, Whh, Why, bh, by], [dL_dWxh, dL_dWhh, dL_Why, dL_dbh, dL_dby]):
        weights -= lr * gradient

    # break
    
    print(f"[Loss]: {loss}")

[Loss]: 0.01055750703674984
[Loss]: 0.010533980088234233
[Loss]: 0.01051058024853176
[Loss]: 0.010487306718386533
[Loss]: 0.010464158686661403
[Loss]: 0.01044113533181351
[Loss]: 0.010418235823263278
[Loss]: 0.010395459322668086
[Loss]: 0.010372804985107486
[Loss]: 0.010350271960177994
[Loss]: 0.010327859393019879
[Loss]: 0.0103055664252564
[Loss]: 0.010283392195873153
[Loss]: 0.010261335842031103
[Loss]: 0.010239396499817616
[Loss]: 0.010217573304941031
[Loss]: 0.010195865393374502
[Loss]: 0.010174271901949587
[Loss]: 0.010152791968903178
[Loss]: 0.01013142473438236
[Loss]: 0.01011016934090768
[Loss]: 0.010089024933802869
[Loss]: 0.010067990661586399
[Loss]: 0.010047065676329535
[Loss]: 0.010026249133986642
[Loss]: 0.010005540194697635
[Loss]: 0.009984938023059503
[Loss]: 0.009964441788381989
[Loss]: 0.009944050664907784
[Loss]: 0.009923763832023137
[Loss]: 0.009903580474441927
[Loss]: 0.00988349978237495
[Loss]: 0.009863520951677033
[Loss]: 0.009843643183983192
[Loss]: 0.009823865686

In [41]:
sample("h", 10)

'ello worll'